In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_true = pd.read_csv('True.csv')
df_false = pd.read_csv('Fake.csv')


In [ ]:
df_true['predict']=1
df_false['predict']=0

In [ ]:
df=pd.concat([df_true,df_false])

In [ ]:
df['fulltext']=df['subject']+" "+df['title']+" "+df['text']

In [ ]:
df.drop(columns=['text', 'title', 'subject','date'], inplace=True)

In [ ]:
import spacy, re
#Data Cleanup

df['fulltext']=df['fulltext'].str.replace('\n','')
df['fulltext']=df['fulltext'].str.replace('\r','')
df['fulltext']=df['fulltext'].str.replace('\t','')

#This removes unwanted texts
df['fulltext'] = df['fulltext'].apply(lambda x: re.sub(r'[0-9]','',x))
df['fulltext'] = df['fulltext'].apply(lambda x: re.sub(r'[/(){}\[\]\|@,;.:-]',' ',x))

#Converting all upper case to lower case
df['fulltext']= df['fulltext'].apply(lambda s:s.lower() if type(s) == str else s)


#Remove un necessary white space
df['fulltext']=df['fulltext'].str.replace('  ',' ')

#Remove Stop words
nlp=spacy.load("en_core_web_sm")
df['fulltext'] =df['fulltext'].apply(lambda x: ' '.join([word for word in x.split() if nlp.vocab[word].is_stop==False ]))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['fulltext'], df.predict, test_size = 0.20, stratify=df.predict, random_state=666)

train_df = pd.DataFrame({0: X_train, 1: y_train})
test_df = pd.DataFrame({0: X_test, 1: y_test})

In [ ]:
!pip install transformers

In [ ]:
!pip install simpletransformers

In [ ]:
from simpletransformers.classification import ClassificationModel


model = ClassificationModel('bert', 'bert-base-uncased', num_labels=2, args={'overwrite_output_dir': True, "train_batch_size": 32, "save_steps": 10000, "save_model_every_epoch": False,
                                                                           'num_train_epochs': 5}, use_cuda=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This b

In [ ]:
model.train_model(train_df)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/71 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:882: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 5:   0%|          | 0/1123 [00:00<?, ?it/s]

...

Running Epoch 2 of 5:   0%|          | 0/1123 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/1123 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/1123 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/1123 [00:00<?, ?it/s]

(5615, 0.007543494570761148)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/17 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/90 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [ ]:
import numpy as np
preds = [np.argmax(tuple(m)) for m in model_outputs]

In [ ]:
from sklearn.metrics import f1_score

print(f1_score(test_df[1], preds, average='micro'))
print(f1_score(test_df[1], preds, average='macro'))

1.0
1.0


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(test_df[1], preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4696
           1       1.00      1.00      1.00      4284

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980

